# Unpacking Sequences — Advanced

This notebook goes beyond basic tuple/list/string unpacking. You'll learn:
- Starred (extended) unpacking and common idioms (`head, *body, tail`, throwaways with `_`)
- Nested unpacking (including with `dict.items()`)
- Unpacking *any* iterable (generators), and caveats (e.g., sets)
- Swapping, parallel updates, and loop unpacking patterns
- Argument unpacking for calls (`*iterable`, `**mapping`) and merging with literals
- Edge cases, error messages, and performance notes


## 1) Quick recap: straight unpacking

In [1]:
pair = (5.0, 5.12)  # (APR, APY)
apr, apy = pair
apr, apy

(5.0, 5.12)

Unpacking works with any iterable (lists, tuples, strings, generators, ...),
as long as the number of targets matches the number of values (unless using a starred target).

In [2]:
x, y, z = 'abc'
(x, y, z)

('a', 'b', 'c')

## 2) Starred (extended) unpacking
Use `*name` to capture “the rest”. You may use at most one starred target per assignment.

In [3]:
data = [10, 20, 30, 40, 50]
head, *body = data
head, body  # body becomes a list

(10, [20, 30, 40, 50])

In [4]:
*prefix, tail = data
prefix, tail

([10, 20, 30, 40], 50)

In [5]:
front, *middle, back = data
front, middle, back

(10, [20, 30, 40], 50)

**Throwaways**: use `_` or `__` when you intentionally ignore values (convention only).

In [6]:
_first, *_ignored, last = range(7)
last

6

## 3) Nested unpacking
Targets can be nested to mirror the shape of the right-hand side.

In [22]:
record = (
    42,
    (3.0, 4.0, 5.0),
    ("python", "unpacking")
)
rec_id, (x, y, z), (t1, t2) = record
rec_id, x, y, z, t1, t2

(42, 3.0, 4.0, 5.0, 'python', 'unpacking')

You can also unpack while iterating dictionaries using `.items()`:

In [8]:
person = {"name": "Ada", "born": 1815}
pairs = []
for k, v in person.items():  # k, v unpack each (key, value) tuple
    pairs.append((k, v))
pairs

[('name', 'Ada'), ('born', 1815)]

## 4) Unpacking *any* iterable (including generators)
Sets are iterable but unordered; their iteration order is not guaranteed.
Generators produce values on-the-fly and are single-use.

In [9]:
def squares(n):
    for i in range(n):
        yield i * i

a, b, c = squares(3)
a, b, c

(0, 1, 4)

With starred targets you can collect an arbitrary number of items from an iterable of unknown length:

In [10]:
*most, last = squares(6)
most, last

([0, 1, 4, 9, 16], 25)

## 5) Classic patterns: swap, head/tail, CSV rows, enumerate
**Swap without a temp** works because the RHS tuple is fully evaluated before assignment happens.

In [11]:
a, b = 100, 3.14
a, b = b, a
a, b

(3.14, 100)

In [12]:
# Parsing 'CSV' row
row = "ada,1815,analytical engine"
name, year, topic = row.split(",")
name, int(year), topic

('ada', 1815, 'analytical engine')

In [13]:
# enumerate returns (index, item) pairs, easy to unpack in loops
letters = list("abcd")
indexed = [(i, ch) for i, ch in enumerate(letters, start=1)]
indexed

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]

## 6) Argument unpacking for function calls and merging literals
- `*seq` unpacks a sequence into positional arguments
- `**mapping` unpacks a dict into keyword arguments
- Inside literals you can merge with `*` (lists/tuples/sets) and `**` (dicts)

In [14]:
def area(w, h, *, unit="px"):
    return {"area": w*h, "unit": unit}

args = (8, 5)
kw = {"unit": "cm^2"}
area(*args, **kw)

{'area': 40, 'unit': 'cm^2'}

In [15]:
a = [1, 2]
b = (3, 4)
c = {5, 6}
merged_list = [*a, *b, *c]  # order: left-to-right iteration
merged_tuple = (*a, *b, *c)
d1 = {"x": 1, "y": 2}
d2 = {"y": 20, "z": 3}
merged_dict = {**d1, **d2}  # later keys override earlier
merged_list, merged_tuple, merged_dict

([1, 2, 3, 4, 5, 6], (1, 2, 3, 4, 5, 6), {'x': 1, 'y': 20, 'z': 3})

## 7) Path / filesystem example (tuple-of-parts)
Unpacking plays well with libraries that expose iterable views (like `pathlib.Path.parts`).

In [16]:
from pathlib import Path
p = Path("/usr/local/bin/python3")
root, *folders, filename = p.parts
root, folders, filename

('\\', ['usr', 'local', 'bin'], 'python3')

## 8) Edge cases & informative errors
Unpacking count must match unless a single starred target absorbs the rest. Only one starred target is allowed at a given assignment level.

In [17]:
try:
    a, b = (1, 2, 3)
except ValueError as e:
    err1 = str(e)
try:
    a, b, c = (1, 2)
except ValueError as e:
    err2 = str(e)
err1, err2

('too many values to unpack (expected 2)',
 'not enough values to unpack (expected 3, got 2)')

In [18]:
try:
    *a, *b = [1, 2, 3]
except SyntaxError as e:
    pass
# Note: This is a SyntaxError at parse time; cannot be caught at runtime in a cell.
# The rule: only one starred target allowed per assignment level.

SyntaxError: multiple starred expressions in assignment (2449514481.py, line 2)

## 9) Parallel updates in place
You can update multiple items of a list (or variables) in one statement using unpacking on both sides.

In [19]:
nums = [10, 20, 30, 40]
nums[0], nums[-1] = nums[-1], nums[0]
nums

[40, 20, 30, 10]

## 10) Unpacking with `heapq` / priority queues
Common in algorithms: unpack `(priority, payload)` pairs as you pop/push.

In [20]:
import heapq
h = []
for pr, item in [(3, 'C'), (1, 'A'), (2, 'B')]:
    heapq.heappush(h, (pr, item))
out = []
while h:
    priority, payload = heapq.heappop(h)
    out.append((priority, payload))
out

[(1, 'A'), (2, 'B'), (3, 'C')]

## 11) Performance notes
Unpacking itself is very fast. The heavy part is often constructing the RHS (e.g., building lists) or materializing generators. Here's a tiny, noisy micro-benchmark (indicative only).

In [21]:
from timeit import timeit
setup = "data = list(range(1000))"
t_unpack_front = timeit("a, *rest = data", setup=setup, number=20000)
t_slice_front  = timeit("a, rest = data[0], data[1:]", setup=setup, number=20000)
{"unpack_front": t_unpack_front, "slice_front": t_slice_front}

{'unpack_front': 0.16184960000100546, 'slice_front': 0.09978879999835044}

## 12) Summary
- Use **starred unpacking** to flexibly capture the “rest”.
- Mirror nested shapes to get values you need (including from `dict.items()`).
- Unpack generators with care (single-use) and avoid relying on set order.
- Use unpacking for clean swaps, parallel updates, and function calls (`*`/`**`).
- Remember the one-star rule and the helpful error messages when counts don’t match.